In [1]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel, CLIPImageProcessor

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

image = Image.open("/home/borth/sketch2shape/data/siamese_chair_medium/1a74a83fa6d24b3cacd67ce2c72c02e/images/00000.jpg")

inputs = processor(text=[""], images=[image, image], return_tensors="pt", padding=True)
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [2]:
inputs.data["pixel_values"].mean()


tensor(1.7172)

In [9]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=[""], images=[image, image], return_tensors="pt", padding=True)
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

CLIPOutput(loss=None, logits_per_image=tensor([[16.2034],
        [16.2034]], grad_fn=<TBackward0>), logits_per_text=tensor([[16.2034, 16.2034]], grad_fn=<MulBackward0>), text_embeds=tensor([[ 9.8798e-03, -1.8297e-03,  1.9789e-02, -2.5857e-02,  1.9348e-03,
          3.9474e-03,  6.0765e-03, -1.4420e-02,  1.6274e-02, -1.9260e-02,
         -1.6453e-03, -2.5102e-03, -2.3597e-02,  1.1575e-02, -1.0197e-02,
          3.7195e-03,  1.4770e-02,  9.2950e-03, -7.1520e-03,  1.6400e-03,
          1.1668e-02, -5.7529e-03, -3.5134e-03, -3.4461e-03,  9.8475e-03,
         -2.4146e-02,  1.7663e-02,  3.7820e-03,  3.7147e-02,  1.0968e-02,
         -1.3619e-02, -7.2178e-03, -1.9867e-02, -1.2346e-02, -1.6044e-02,
          1.8504e-02,  4.6477e-03, -2.4104e-02, -8.4608e-03, -2.4040e-02,
         -5.6259e-03, -5.6369e-03,  7.0979e-03,  4.5978e-03,  1.1762e-02,
         -2.1662e-02,  4.2041e-03, -4.9361e-03, -1.3477e-02, -2.2331e-02,
         -2.3357e-03, -7.8273e-03,  6.1767e-03, -4.4492e-03,  3.4784e-03,
   

: 

In [13]:
import matplotlib.pyplot as plt
from lib.utils import load_config
from lib.models.siamese import Siamese
from lightning import Trainer
from lib.eval.tester import SiameseTester
from pathlib import Path
from lib.data.siamese_datamodule import SiameseDataModule
import hydra
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from transformers import CLIPProcessor, CLIPModel

cfg = load_config("eval_siamese", overrides=["+experiment=eval"])
cfg.ckpt_path = "/home/borth/sketch2shape/logs/train/runs/2023-12-10_20-29-37/checkpoints/last.ckpt"
model = Siamese.load_from_checkpoint(cfg.ckpt_path)
tester = SiameseTester(model=model.decoder)
datamodule = hydra.utils.instantiate(cfg.data, train=False)
datamodule.setup("all")

        

/home/borth/miniconda3/envs/pytorch3d/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'decoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['decoder'])`.
/home/borth/miniconda3/envs/pytorch3d/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'miner' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['miner'])`.
/home/borth/miniconda3/envs/pytorch3d/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.


In [12]:
class EvalCLIP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding_size = 768
        self.model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
    
    def forward(self, batch):
        inputs = processor(text=[""], images=batch, return_tensors="pt", padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.image_embeds

model = EvalCLIP()
tester = SiameseTester(model=model)
trainer = Trainer(
    accelerator="gpu",
    max_epochs=1,
    enable_checkpointing=False,
)
trainer.validate(
    tester,
    dataloaders=[
        datamodule.test_dataloader(),
        datamodule.val_dataloader(),
    ],
)


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/borth/miniconda3/envs/pytorch3d/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorb

Validation: |          | 0/? [00:00<?, ?it/s]

/home/borth/miniconda3/envs/pytorch3d/lib/python3.9/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
